# IA Jojo

## Introdução

## Sumário
1. [Instalação de Bibliotecas](#Instalação-de-Bibliotecas)

## Instalação de Bibliotecas

Primeiramente, se ainda não foi feito, instalaremos todas as bibliotecas necessárias para esse projeto:


In [ ]:
%pip install -r requirements.txt